In [1]:
import json
import random
import re
import time

from dataclasses import dataclass
from datetime import datetime
from itertools import islice

import pandas as pd

from gretel_client import Gretel, configure_session


In [10]:
NUM_DOMAINS = 10
NUM_SEEDS = 250
NUM_PROMPTS_PER_SEED = 5
GRETEL_API_KEY = "TOKEN HERE"

In [11]:
# Domain prompt
DOMAIN_PROMPT = f"""
You are a data expert across all of the major industries/verticals. Generate a diverse dataset
of domains and detailed descriptions of data seen in these domains.

For example:
"healthcare": "Comprehensive data on patient demographics, medical histories, treatment protocols, and outcomes; drug efficacy studies and side effects.",
"education": "Detailed records on student demographics, academic performance, curriculum efficacy, school management, and educational resource allocation.",
"retail": "Extensive data on product sales, customer demographics, inventory levels, supplier information, and consumer buying patterns and preferences.",
"finance": "Detailed financial data including stock market trends, transaction records, investment portfolios, risk assessment models, and banking operations.",
"real estate": "Real estate market data covering property listings, pricing trends, property features, regional market analysis, and real estate investment returns."

##### GUIDELINES
The dataset should contain only four columns
Column 1: domain
Column 2: domain_description
"""

# Generate domain descriptions
try:
    gretel = Gretel(api_key=GRETEL_API_KEY)
    tabl1m = gretel.factories.initialize_navigator_api("tabular", backend_model='gretelai/auto')
    tabl1m.request_timeout_sec = 60

    domain_descriptions_pd = tabl1m.generate(prompt=DOMAIN_PROMPT, num_records=NUM_DOMAINS, as_dataframe=True,
                                             top_p=0.92, temperature=0.8, stream=False)
    domain_descriptions = {k.replace('-', '_'): v for k, v in zip(domain_descriptions_pd['domain'], domain_descriptions_pd['domain_description'])}
    domain_descriptions = dict(islice(domain_descriptions.items(), NUM_DOMAINS))
except Exception as e:
    print(f"Error generating table: {e}")


Backend model: gretelai/auto
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀


Generating records: 100%|██████████| 10/10 [00:06, 1.61 records/s]


In [12]:
# User proficiencies
user_proficiencies = {
    "beginner",
    "intermediate",
    "advanced",
    "expert"
}

# SQL complexities
sql_complexity = {
    "basic SQL with a simple select statement",
    "only one join (specify inner, outer, cross)",
    "two or more joins (specify inner, outer, cross)",
    "subqueries, including correlated and nested subqueries",
    "common table expressions",
    "aggregation functions (COUNT, SUM, AVG, MIN, MAX, etc.), and HAVING clause",
    "set operations such as UNION, INTERSECT, and EXCEPT",
    "window functions (e.g., ROW_NUMBER, LEAD, LAG, RANK, NTILE, PERCENT_RANK, etc.) with partitioning and ordering",
    "pivoting and unpivoting"
}

# SQL tasks
sql_task = {
    "data definition: creating, altering, or dropping tables and other database objects",
    "data retrieval: basic data fetching queries",
    "data manipulation: inserting, updating, or deleting records",
    "analytics and reporting: generating reports, dashboards, and analytical insights",
    "transactional processing: SQL transaction control statements (e.g., BEGIN, COMMIT, ROLLBACK)"
}


In [13]:
# Generate contextual tag data
sampled_contextual_tag_data = [
    (
        domain,
        domain_descriptions[domain],
        random.choice(list(user_proficiencies)),
        random.choice(list(sql_complexity)),
        random.choice(list(sql_task))
    )
    for _ in range(NUM_SEEDS)
    for domain in [random.choice(list(domain_descriptions.keys()))]
]
# Convert sampled data to a DataFrame
contextual_tags = pd.DataFrame(
    sampled_contextual_tag_data,
    columns=["domain", "data_description", "user_level", "sql_complexity", "sql_task"]
)
print(f'Created {len(contextual_tags)} contextual tag permutations')
print(contextual_tags.head())


Created 250 contextual tag permutations
           domain                                   data_description  \
0     real estate  Real estate market data covering property list...   
1          retail  Extensive data on product sales, customer demo...   
2  transportation  Extensive data on fleet management, logistics,...   
3   manufacturing  Detailed records on production processes, inve...   
4     agriculture  Detailed data on crop yields, farming practice...   

     user_level                                     sql_complexity  \
0      advanced           basic SQL with a simple select statement   
1  intermediate  aggregation functions (COUNT, SUM, AVG, MIN, M...   
2        expert  set operations such as UNION, INTERSECT, and E...   
3        expert    two or more joins (specify inner, outer, cross)   
4      beginner                           common table expressions   

                                            sql_task  
0  analytics and reporting: generating reports, d..

In [14]:
# Function to generate text to SQL data
def generate_text2sql_data(domain: str, data_description: str, user_level: str, sql_complexity: str, sql_task: str, verbose=False) -> str:
    PROMPT = f"""
    You are a data and SQL expert. Generate a diverse dataset using the following guidelines

    ##### GUIDELINES
    The dataset should contain only four columns
    Column 1: natural language prompt
        * Column name: prompt
        * a well-formulated question or command in everyday English, representing a user query to a database
        * prompt should require using {user_level} level of SQL and ideally {sql_complexity}
        * prompt should be in the {domain} domain and pertain to {data_description}
        * prompt should pertain to {sql_task} first and foremost
    Column 2: tables and views that already exist in the database
        * Column name: context
        * this should be database context, NOT the SQL query representing a user's prompt
        * include complete executable SQL table CREATE statements and/or view CREATE statements with capitalized keywords
        * only table CREATE and CREATE + INSERT statements are allowed as database context. No ALTER/DROP/UPDATE
        * provide up to five tables/views that are relevant to the user's natural language prompt
        * provide only the SQL code for create statements, separated by a semicolon
        * make sure there is no text preceding or following SQL code
        * do not use newline characters or breakline tags in SQL code
        * do not use ellipsis anywhere: this should be a valid, executable SQL statement
        * do not use phrases like "same as previous example"
        * table names and schemas should correspond to the {domain} domain and focus on {data_description}
        * make sure there is no text of any kind preceding or following SQL code
    Column 3: SQL query
        * Column name: sql
        * A complete and executable SQL query used to answer/execute the natural language prompt
        * do not use ellipsis anywhere: this should be a valid, executable SQL statement
        * SQL should be based on the database context generated above
        * To the extent possible, SQL should leverage {sql_complexity}
        * SQL should be written at an {user_level} SQL proficiency level
        * do not use newline characters or breakline tags in SQL code
        * make sure there is no text of any kind preceding or following SQL code
    Column 4: explanation
        * Column name: explanation
        * a step-by-step explanation of what the SQL query is doing
    """

    if verbose:
        print(PROMPT)

    try:
        gretel = Gretel(api_key=GRETEL_API_KEY)
        tabl1m = gretel.factories.initialize_navigator_api("tabular", backend_model='gretelai-google/gemini-pro')
        tabl1m.request_timeout_sec = 60
        prompt_context = tabl1m.generate(prompt=PROMPT, num_records=NUM_PROMPTS_PER_SEED, as_dataframe=True,
                                         top_p=0.92, top_k=40, temperature=0.8, stream=False)
    except Exception as e:
        return f"Error generating table: {e}"

    # return the pandas dataframe as json so we can then expand in Pandas
    return prompt_context.to_json()


In [15]:
print(gretel.factories.get_navigator_model_list("tabular"))

['gretelai/auto', 'gretelai/Mistral-7B-Instruct-v0.2/industry', 'gretelai-azure/gpt-3.5-turbo', 'gretelai-google/gemini-pro']


In [16]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random

# Function to apply generate_text2sql_data and handle errors
def safe_generate_text2sql_data_with_retry(row, max_retries=2):
    retries = 0
    wait_time = random.uniform(3, 10)
    time.sleep(wait_time)
    while retries <= max_retries:
        try:
            return generate_text2sql_data(row['domain'], row['data_description'], row['user_level'], row['sql_complexity'], row['sql_task'])
        except Exception as e:
            print(f"ERROR: {e}")
            if '403' in str(e) and retries < max_retries:
                print(f"403 ERROR! retry: {retries+1}/{max_retries}")
                retries += 1
                wait_time = random.uniform(3, 10)
                time.sleep(wait_time)
            else:
                return f"error generating: {e}"

# Parallelize the application of generate_text2sql_data
def parallel_apply(df, func, num_workers=4):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(func, row): idx for idx, row in df.iterrows()}
        results = []
        for future in as_completed(futures):
            idx = futures[future]
            try:
                results.append(future.result())
            except Exception as e:
                print(e)
                results.append(f"error generating: {e}")
        return results

In [17]:
test = contextual_tags
test['text2sql_json'] = parallel_apply(test, safe_generate_text2sql_data_with_retry)

Backend model: gretelai-google/gemini-pro
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀


Generating records:   0%|          | 0/5 [00:00,? records/s]

Backend model: gretelai-google/gemini-pro
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀



Generating records:   0%|          | 0/5 [00:00,? records/s]

Backend model: gretelai-google/gemini-pro
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀


...


Generating records:  20%|██        | 1/5 [00:04, 0.20 records/s]


Generating records:  60%|██████    | 3/5 [00:08, 0.39 records/s]

Generating records: 100%|██████████| 5/5 [00:21, 0.23 records/s]

Generating records: 100%|██████████| 5/5 [00:13, 0.36 records/s]

Generating records: 100%|██████████| 5/5 [00:13, 0.36 records/s]


Backend model: gretelai-google/gemini-pro
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀


Generating records:   0%|          | 0/5 [00:00,? records/s]

Backend model: gretelai-google/gemini-pro
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀



Generating records: 100%|██████████| 5/5 [00:12, 0.40 records/s]

Generating records: 100%|██████████| 5/5 [00:16, 0.30 records/s]


In [19]:
test_errors = test[test['text2sql_json'].str.contains('Error generating')]
test2 = test[~test['text2sql_json'].str.contains('Error generating')]
print(f'{len(test2)} records remain after removing errors')

195 records remain after removing errors


In [20]:
test

,domain,data_description,user_level,sql_complexity,sql_task,text2sql_json
0,real estate,Real estate market data covering property list...,advanced,basic SQL with a simple select statement,"analytics and reporting: generating reports, d...",Error generating table: (403)\nReason: Forbidd...
1,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...","{""prompt"":{""0"":""Identify the top five cities i..."
2,transportation,"Extensive data on fleet management, logistics,...",expert,"set operations such as UNION, INTERSECT, and E...",data retrieval: basic data fetching queries,"{""prompt"":{""0"":""Which customers have placed or..."
3,manufacturing,"Detailed records on production processes, inve...",expert,"two or more joins (specify inner, outer, cross)",transactional processing: SQL transaction cont...,"{""prompt"":{""0"":""Find all orders placed in the ..."
4,agriculture,"Detailed data on crop yields, farming practice...",beginner,common table expressions,"data manipulation: inserting, updating, or del...","{""prompt"":{""0"":""Which countries have the highe..."
...,...,...,...,...,...,...
245,manufacturing,"Detailed records on production processes, inve...",intermediate,"set operations such as UNION, INTERSECT, and E...","data manipulation: inserting, updating, or del...","{""prompt"":{""0"":""Which states have the highest ..."
246,agriculture,"Detailed data on crop yields, farming practice...",intermediate,"only one join (specify inner, outer, cross)",transactional processing: SQL transaction cont...,"{""prompt"":{""0"":""What is the average yield of w..."
247,technology,"Comprehensive data on product development, mar...",intermediate,pivoting and unpivoting,"data manipulation: inserting, updating, or del...","{""prompt"":{""0"":""Which products are currently i..."
248,manufacturing,"Detailed records on production processes, inve...",advanced,"subqueries, including correlated and nested su...","data manipulation: inserting, updating, or del...","{""prompt"":{""0"":""How many different products di..."


In [21]:
# Parse JSON
def parse_json(row):
    try:
        return json.loads(row['text2sql_json'])
    except json.JSONDecodeError:
        return {}

test2['parsed_json'] = test2.apply(parse_json, axis=1)

# Transform maps into arrays of keys and values, zip and explode
def explode_json(row):
    data = []
    for key in row['parsed_json']['prompt'].keys():
        data.append({
            "domain": row['domain'],
            "data_description": row['data_description'],
            "user_level": row['user_level'],
            "sql_complexity": row['sql_complexity'],
            "sql_task": row['sql_task'],
            "prompt": row['parsed_json']['prompt'][key],
            "context": row['parsed_json']['context'][key],
            "sql": row['parsed_json']['sql'][key],
            "explanation": row['parsed_json']['explanation'][key]
        })
    return pd.DataFrame(data)

exploded_text2sql_data = pd.concat([explode_json(row) for idx, row in test2.iterrows()], ignore_index=True)

/tmp/ipykernel_5395/3429042814.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['parsed_json'] = test2.apply(parse_json, axis=1)


In [22]:
exploded_text2sql_data.iloc[:1000].to_csv("navigator_data.csv")

In [23]:
exploded_text2sql_data

,domain,data_description,user_level,sql_complexity,sql_task,prompt,context,sql,explanation
0,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...",Identify the top five cities in the United Sta...,CREATE TABLE listings (\nlisting_id INT PRIMAR...,"SELECT city, state, median_price FROM median_p...",This query first creates a view called `median...
1,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...",What is the average price per square foot for ...,CREATE TABLE listings (\nlisting_id INT PRIMAR...,SELECT AVG(price / square_footage) AS average_...,This query calculates the average price per sq...
2,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...",What are the top 5 cities in the United States...,CREATE TABLE US_Cities_Median_Home_Prices (Cit...,"SELECT City, State, Median_Home_Price FROM US_...","This SQL query selects the City, State, and Me..."
3,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...",Show me the average price per square foot for ...,CREATE TABLE US_States_Avg_Price_Per_SqFt (Sta...,"SELECT State, Avg_Price_Per_SqFt FROM US_State...",This SQL query selects the State and Avg_Price...
4,retail,"Extensive data on product sales, customer demo...",intermediate,"aggregation functions (COUNT, SUM, AVG, MIN, M...","data manipulation: inserting, updating, or del...",Find all properties in California that have a ...,"CREATE TABLE CA_Properties (Property_ID INT, A...","SELECT Address, List_Price, Bedrooms FROM CA_P...","This SQL query selects the Address, List_Price..."
...,...,...,...,...,...,...,...,...,...
970,transportation,"Extensive data on fleet management, logistics,...",intermediate,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...","data manipulation: inserting, updating, or del...",Add a new record to the Vehicles table for a N...,CREATE TABLE Vehicles (vehicle_id INT PRIMARY ...,"INSERT INTO Vehicles (make, model, license_pla...",This SQL statement inserts a new row into the ...
971,transportation,"Extensive data on fleet management, logistics,...",intermediate,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...","data manipulation: inserting, updating, or del...",Update the customer's address in the Customers...,CREATE TABLE Customers (customer_id INT PRIMAR...,"UPDATE Customers SET address = '123 Main St, A...",This SQL statement updates the address of the ...
972,transportation,"Extensive data on fleet management, logistics,...",intermediate,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...","data manipulation: inserting, updating, or del...",Delete the record from the Orders table where ...,"CREATE TABLE Orders (order_id INT PRIMARY KEY,...",DELETE FROM Orders WHERE order_id = 789;,This SQL statement deletes the row from the Or...
973,transportation,"Extensive data on fleet management, logistics,...",intermediate,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...","data manipulation: inserting, updating, or del...",Calculate the average mileage of all vehicles ...,CREATE TABLE Vehicles (vehicle_id INT PRIMARY ...,"SELECT make, AVG(mileage) AS average_mileage F...",This SQL statement calculates the average mile...


In [1]:
# Load Finetuned Model

In [1]:
import csv
import yaml
from typing import Tuple
import re
from functools import partial

from datasets import load_dataset
from transformers import AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from peft import AutoPeftModelForCausalLM
import torch

from llmtune.pydantic_models.config_model import Config

In [2]:
EXPERIMENT_HASH = "eHP6Vw"
config_path = f"./experiment/{EXPERIMENT_HASH}/config/config.yml"

with open(config_path, "r") as f:
    config = yaml.safe_load(f)
    config = Config(**config)

In [3]:
weights_path = f"./experiment/{EXPERIMENT_HASH}/weights/"
model = AutoPeftModelForCausalLM.from_pretrained(
    weights_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
model = model.merge_and_unload()
tok = AutoTokenizer.from_pretrained(weights_path, device_map="auto")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:
def infer(prompt: str, model, tok: AutoTokenizer) -> str:
    """
    Outputs predicted sequence and probability
    """
    input_ids = tok.encode(prompt, return_tensors="pt", truncation=True).cuda()
    gen_config = GenerationConfig(
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.2,
        pad_token_id=tok.pad_token_id,
        return_dict_in_generate=True,
        output_scores=True,
        )

    gen_results = model.generate(input_ids, gen_config)
    gen_tok_ids = gen_results.sequences.squeeze(0)[input_ids.shape[1]:]

    seq = tok.decode(gen_tok_ids, skip_special_tokens=True)

    seq_score = torch.stack(gen_results.scores).squeeze(1) # get raw output scores 
    seq_score = torch.log(torch.softmax(seq_score, dim=1)) # softmax and log to get log_prob
    seq_score = seq_score.gather(dim=1, index=gen_tok_ids.view(-1, 1)).squeeze()
    seq_score = torch.exp(torch.sum(seq_score)).item() # sum and exp to get prob

    return seq, seq_score

In [ ]:
with open("prompts.csv", mode='r', newline='') as file:
    reader = csv.reader(file)
    headers = next(reader)  # Read the header
    data = [tuple(row) for row in reader]  # Read the data and convert each row to a tuple

processed_preds = {}
for idx, row in enumerate(data):
    out, _ = infer(row[1], model, tok)
    out = out.strip()
    out += f"\t----- bird -----\t{data[idx][0]}"
    processed_preds[str(idx)] = out
    print(f"Done {idx+1}/{len(data)}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Done 1/500
Done 2/500
Done 3/500
Done 4/500
Done 5/500
Done 6/500
Done 7/500
Done 8/500
Done 9/500
Done 10/500
Done 11/500
Done 12/500
Done 13/500
Done 14/500
Done 15/500
Done 16/500
Done 17/500
Done 18/500
Done 19/500
Done 20/500
Done 21/500
Done 22/500
Done 23/500
Done 24/500
Done 25/500
Done 26/500
Done 27/500
Done 28/500
Done 29/500
Done 30/500
Done 31/500
Done 32/500
Done 33/500
Done 34/500
Done 35/500
Done 36/500
Done 37/500
Done 38/500
Done 39/500
Done 40/500
Done 41/500
Done 42/500
Done 43/500
Done 44/500
Done 45/500
Done 46/500
Done 47/500
Done 48/500
Done 49/500
Done 50/500
Done 51/500
Done 52/500
Done 53/500
Done 54/500
Done 55/500
Done 56/500
Done 57/500
Done 58/500
Done 59/500
Done 60/500
Done 61/500
Done 62/500
Done 63/500
Done 64/500
Done 65/500
Done 66/500
Done 67/500
Done 68/500
Done 69/500
Done 70/500
Done 71/500
Done 72/500
Done 73/500
Done 74/500
Done 75/500
Done 76/500
Done 77/500
Done 78/500
Done 79/500
Done 80/500
Done 81/500
Done 82/500
Done 83/500
Done 84/500
D

In [6]:
import json
with open("predict_mini_dev_mistral-navigator_SQLite.json", 'w') as json_file:
    json.dump(processed_preds, json_file, indent=4)

In [ ]:
processed_preds